# Inferential statistics
## Part III - Inferential Analysis

We're now going to look for answers to the ongoing basketball discussions between you and your family. The main ones we want to reasearch are the following:

- Your grandmother says that your sister couldn't play in a professional basketball league (not only the WNBA, but ANY professional basketball league) because she's too skinny and lacks muscle.
- Your sister says that most female professional players fail their free throws.
- Your brother-in-law heard on the TV that the average assists among NBA (male) and WNBA (female) players is 52 for the 2016-2017 season. He is convinced this average would be higher if we only considered the players from the WNBA.

Let's investigate these claims and see if we can find proof to refute or support them.

### Libraries
Import the necessary libraries first.

In [1]:
# Libraries
import math
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
pd.set_option('max_columns', 50)

### Load the dataset

Load the cleaned dataset.

In [2]:
#your code here

wnba = pd.read_csv('wnba_clean.csv')

# Question 1: Can my sister play in a professional female basketball league?

As we said, you grandmother is convinced that your sister couldn't play in a professional league because of her physique and weight (her weight is 67kg). 

To find an actual answer to the question we first need to know what's the average weight of a professional female basketball player. The data we have only refers to the WNBA league and not to every female professional basketball league in the world, therefore we have no way of actually calculating it.

Still, given that we do have *some* data we can **infer** it using a sample of players like the one we have. 

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the average weight. Do you feel it actually fulfills those requirements? Do you need to make any assumptions? We could calculate a confidence interval to do the inference, but do you know any other ways?**

In [11]:
# your answer here

# i would use the scipy package
# we need to have the mean, the std and the targeted weight.
# we fulfill
# we can also see if a hypothesis is true.

**Now that all the requirements have been taken into account, compute the confidence interval of the average weight with a confidence level of 95%.**

In [4]:
# your code here

mu = wnba['Weight'].mean()
sigma = wnba['Weight'].std()

stats.norm.interval(0.95, loc=mu, scale=sigma)

(57.42689286914671, 100.53085360972653)

In [6]:
"""
H0: mu > 67
H1: mu <= 67

5% confidence
"""

ttest_1samp(wnba['Weight'],67)

Ttest_1sampResult(statistic=12.981385575989544, pvalue=7.635325397241363e-26)

**What can you say about these results?**

In [10]:
#your-answer-here

# the range of the weight is very high, going from 57 to 100.

**If your sister weighs 67kg what would you tell your grandmother in regards to her assumption?**

In [9]:
#your-answer-here

# she can play for sure.

## Bonus: Can you plot the probability distribution of the average weight, indicating where the critical region is?

In [8]:
# your code here

# Question 2: Do female professional basketball players fail the majority of their free throws?

You do not agree with your sister when she says that most female players fail their free throws. You decide to try and estimate the percentage of players that fail more than 40% of their free throws using, you guessed it, the WNBA sample.

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the proportion of players that miss more than 40% of their free throws. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [50]:
# your answer here

# first i need to see the proportion of players that miss more than 40% of their free throws.

trials = len(wnba['FT%'])

Count = wnba[wnba['FT%']<60]

In [51]:
Count

,Unnamed: 0,Name,Team,Pos,Height,Weight,BMI,Birth_Place,Birthdate,Age,College,Experience,Games Played,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TO,PTS,DD2,TD3
11,11,Alyssa Thomas,CON,F,188,84,23.766410,US,"December 4, 1992",24,Maryland,3,28,833,154,303,50.8,0,3,0.0,91,158,57.6,34,158,192,136,48,11,87,399,4,0
14,14,Aneika Henry,ATL,F/C,193,87,23.356332,JM,"February 13, 1986",31,Florida,6,4,22,4,4,100.0,0,0,0.0,0,0,0.0,0,4,4,1,2,0,3,8,0,0
32,32,Cayla George,PHO,C,193,87,23.356332,AU,"April 20, 1987",30,Georgia,1,28,365,40,105,38.1,13,45,28.9,7,12,58.3,10,71,81,15,9,11,13,100,1,0
35,35,Clarissa dos Santos,SAN,C,185,89,26.004383,BR,"October 3, 1988",28,Brazil,4,7,52,8,14,57.1,1,1,100.0,0,0,0.0,3,7,10,7,1,1,5,17,0,0
36,36,Courtney Paris,DAL,C,193,113,30.336385,US,"September 21, 1987",29,Oklahoma,7,16,217,32,57,56.1,0,0,0.0,6,12,50.0,28,34,62,5,6,8,18,70,0,0
47,47,Elizabeth Williams,ATL,F/C,191,87,23.848030,US,"June 23, 1993",24,Duke,3,30,377,48,96,50.0,0,1,0.0,32,55,58.2,35,61,96,5,5,4,21,128,0,0
56,56,Glory Johnson,DAL,F,191,77,21.106878,US,"July 27, 1990",27,Tennessee,4,4,42,3,9,33.3,3,6,50.0,0,0,0.0,0,3,3,1,0,0,4,9,0,0
80,80,Kia Vaughn,NY,C,193,90,24.161722,US,"January 24, 1987",30,Rutgers,9,23,455,62,116,53.4,0,0,0.0,10,19,52.6,39,71,110,16,8,9,21,134,1,0
84,84,Lanay Montgomery,SEA,C,196,96,24.989588,US,"September 17, 1993",23,West Virginia,R,7,28,3,7,42.9,0,0,0.0,0,0,0.0,0,5,5,0,1,4,2,6,0,0
90,90,Maimouna Diarra,LA,C,198,90,22.956841,SN,"January 30, 1991",26,Sengal,R,9,16,1,3,33.3,0,0,0.0,1,2,50.0,3,4,7,1,1,0,3,3,0,0


**Now that all the requirements have been taken into account, compute the confidence interval of the proportion with a confidence level of 95%:**

In [48]:
# your code here

from statsmodels.stats import proportion as prp

prp.proportion_confint(len(Count),nobs=trials,alpha=1-0.95,method='normal')

(0.049558988317297346, 0.14762411027425193)

**What can you comment about our result? What would you tell your sister?**

In [15]:
#your-answer-here

# Saying that most of the female players fail their free throws is too far from a strech, 
# because only 4% to 14% miss more than 40%

# Bonus: Can you plot the probability distribution of the proportion of missed free throws, indicating where the critical region is?

In [ ]:
#your code here

# Question 3: Is the average number of assists for WNBA players only higher than the average for WNBA and NBA players together?

Your brother-in-law is convinced that the average assists for female professional players is higher than the average of both female and male players combined (which is 52 for the 2016-2017 season). You would like to actually prove if this is true or not but you remember your stats teacher saying "you can't *prove* anything, you just can say that *you are not* saying foolishness".

**How would you do it? Try and think about the requirements that your sample must satisfy in order to do that. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [16]:
#your-answer-here

# i would do the two-tailed test to see if it proves right based on the stats.

**Use a two-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [56]:
#your code here
"""
H0: mu >= 52
H1: mu < 52

5% confidence
"""

ttest_1samp(wnba['AST'],52)



Ttest_1sampResult(statistic=-2.1499947192482898, pvalue=0.033261541354107166)

In [18]:
#your-answer-here

# he is foolish

**Now use a one-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [57]:
#your-answer-here

a,b = ttest_1samp(wnba['AST'],52)

b/2


0.016630770677053583

# Bonus: Can you plot the resulting t-distribution of both tests? Indicate where the is the critical region and where does your statistic fall.**

In [ ]:
#your code here

# Bonus: Satisfying your curiosity

You finally managed to solve your family's debates over basketball! While you were doing that you started to take an interest in the normal distribution.

You read that the normal distribution is present in a lot of natural phenomenons, like blood pressure, IQ, weight and height. If, for example, we could plot the distribution of the weights of every human on the planet right now it would have the shape of a normal distribution.

In light of this you would like to see if it's possible to check if the distribution of the weights of the WNBA players is a sample distribution that comes from a population that has a normal distribution, because theoretically this should be the case.

**How would you try to demonstrate that our sample fits a normal distribution? What kind of test would you use? Would you have to make any assumptions?**

In [22]:
#your-answer-here

In [19]:
# your code here

**What are your comments in regards to the results of the test?**

In [24]:
#your-answer-here